### Imports

In [41]:
import numpy as np
import scipy
import pandas as pd
import sklearn as sk
import time
import math
from math import pi

import matplotlib.pyplot as plt

In [4]:
avg_work_hrs_yr = 40 * 52
print(avg_work_hrs_yr)

2080


In [10]:
# hours = pd.readcsv('work_hours.csv')
# hr_per_week = pd.readcsv('work_week.csv')

# set hourly rate
# rates = pd.readcsv('hourly_rate.csv')

# deduction = pd.readcsv('deductions.csv')

hours = np.loadtxt("work_hours.txt")
hours_per_week = np.loadtxt("work_week.txt")
hourly_rates = np.loadtxt("hourly_rate.txt")
pre_tax_deduction = np.loadtxt("pre_tax_deductions.txt")

# salary = np.loadtxt("annual_salary.txt")

OSError: work_hours.txt not found.

### Deductions

##### Need to make updatable via outside txt file

In [90]:
def marital_status(string):
    pass

In [91]:
def deductions_from_taxable_income():
    for item in pre_tax_deductions:
        pass #increases the total amount to be deducted by the new item.

In [93]:
# tax_brackets = pd.read_excel("tax_brackets.xlsx")
# Use the above to update the bracket estimation

def tax_bracket_estimation(total_gross_income):
    if 0.00 < total_gross_income <= 9700:
        tax_bracket = (total_gross_income - deductions()) * 0.10
        tax_bracket = round(tax_bracket, 2)
        return tax_bracket
    elif 9700 < total_gross_income <= 39475:
        tax_bracket = (total_gross_income - marital_status() - deductions()) * 0.12
        tax_bracket = round(tax_bracket, 2)
        return tax_bracket
    elif 39475 < total_gross_income <= 84200:
        tax_bracket = (total_gross_income - marital_status() - deductions()) * 0.22
        tax_bracket = round(tax_bracket, 2)
        return tax_bracket
    elif 84200 < total_gross_income <= 160725:
        tax_bracket = (total_gross_income - marital_status() - deductions()) * 0.24
        tax_bracket = round(tax_bracket, 2)
        return tax_bracket
    elif 160725 < total_gross_income < 204100:
        tax_bracket = (total_gross_income - marital_status() - deductions()) * 0.32
        tax_bracket = round(tax_bracket, 2)
        return tax_bracket
    elif 204100 < total_gross_income < 510300:
        tax_bracket = (total_gross_income - marital_status() - deductions()) * 0.35
        tax_bracket = round(tax_bracket, 2)
        return tax_bracket
    else:
        tax_bracket = (total_gross_income - marital_status() - deductions()) * 0.37
        tax_bracket = round(tax_bracket, 2)
        return tax_bracket

### Compute gross income

###### temporary test values

In [56]:
hours = 40
hourly_rates = 20.92
pre_tax_deductions = 300

###### end temporary test values

In [57]:
wage = [hours * rates, hours * rates]

total_gross_income = np.sum(wage) - 40 * rates

print(wage)
print(total_gross_income)

[836.8000000000001, 836.8000000000001]
836.8000000000001


##### Pay Period Cycling

In [103]:
def awaiting_next_pay_period(pay_schedule):
    # sleep(seconds per minute * minutes per hour * hours per day * days per week * weeks per period)
    if pay_schedule == 1:
        time.sleep(60*60*24*7*1)
        wage.append(hours*rates)
        
    elif pay_schedule == 2:
        time.sleep(60*60*24*7*2)
        wage.append(hours*rates)
    
    elif pay_schedule == 3:
        time.sleep(60*60*24*7*3)
        wage.append(hours*rates)
    
    elif pay_schedule == 4:
        time.sleep(60*60*24*7*4)
        wage.append(hours*rates)
        
#TODO update for a "twice per month" setting to include those who would be paid on two fixed calendar dates

### Projected Annual Income

In [104]:
tax_bracket = 0.22

In [98]:
projected_annual_income = (avg_work_hrs_yr * rates) - tax_bracket * (avg_work_hrs_yr * rates) 

# projected_annual_income = projected_annual_income - tax_deduction * projected_annual_income
print(projected_annual_income)

33940.60800000001


### Compute net income

In [101]:
net_income_01 = total_gross_income - tax_bracket * total_gross_income
print(net_income_01)

652.7040000000001


### Compute expenses

In [102]:
# bills_01 = pd.read_excel('bills01.xlsx')

bills_01 = np.loadtxt('bills01.txt')
expenses_01 = np.sum(bills_01)

OSError: bills01.txt not found.

### Compute remaining income

In [ ]:
rem_income = net_income01 - expenses_01

rem_ann_income = proj_ann_income - rem_income

### Linear regression plot

In [ ]:
i = np.array([1:len(bills_01)])
m = sum((i - np.average(i)) @ bills_01) / sum((i - np.average(i)) ** 2)
c = np.average(bills_01) - m * np.average(i)

# Average spending per data point in bills
# TODO	convert to average spending per day
avg_per_day = np.average(bills_01, axis = 0)

# General disparity
despair_ity = (proj_ann_income - rem_income) / proj_ann_income


### View the budget

In [2]:
# Plot Data
def budget_image():
    visual_expense = pd.read_excel("bills_01.xlsx")
    #Change "person 1" and "person 2" to draw names from spreadsheet
    data = pd.DataFrame([visual_expense], index = ["person 1", "person 2"])
    
    features = list(data)
    number_of_features = len(features)
    
    values_for_person_one = data.iloc[0].tolist()
    values_for_person_one += values_for_person_one[:1]
    
    values_for_person_two = data.iloc[1].tolist()
    values_for_person_two += values_for_person_two[:1]
    
    angles = [n / float(number_of_features) * 2 * pi for n in range(number_of_features)]
    angles += angles[:1]
    
    ax = plt.subplot(111, polar = True)
    
    plt.xticks(angles[:-1], features)
    
    ax.plot(angles, values_for_person_one)
    ax.plot(angles, values_for_person_two)
    
    ax.fill(angles, values_for_person_one, '00e5ee', alpha = 0.1)
    axis.set_title("Person 1")
    plt.show()
    
    ax.fill(angles, values_for_person_two, 'b57edc', alpha = 0.1)
    axis.set_title("Person 2")
    plt.show()
    
    
#    Pie chart
#    labels = 'Rent', 'Water', 'Power', 'Retirement', 'Health Care', 'Car Insurance', 'Rent Insurance', 'PAY PERIOD'
#    sizes = [rental, wat_bill, pow_bill, ret_ded, health_ded, car_ins, rent_ins, PAY_PERIOD()]
#    explode = (0, 0, 0, 0, 0, 0, 0, 0.1)

#    fig1, ax1 = plt.subplots()
#    ax1.pie(sizes, explode = explode, labels = labels, autopct = '%1.2f%%',
#            shadow = True, startangle = 90)
#    ax1.axis('equal') # aspect ratio to make it a circle
#    plt.show()

##### Budget view function

In [ ]:
def view_budget(person, data):    
    
#   features = list(pd.DataFrame([pd.read_excel("bills_01.xlsx")], index = ["person 1", "person 2"]))
    features = ["Rent", "Water", "Power", "Entertainment", "Health Care", "Insurance: Car", "Insurance: Housing", "Else"]
    
    data += data [:1]
    
    angles = [n / 6 * 2 * pi for n in range(6)]
    angles += angles [:1]
    
    ax = plt.subplot(111, polar=True)

    plt.xticks(angles[:-1], features)
    ax.plot(angles, data)
    ax.fill(angles, data, 'blue', alpha=0.1)

    ax.set_title(person)
    plt.show()
    
    

###### Gradient descent: May not use this edition and may revert to the one in the other file.

In [ ]:
def gradientDescent(x, y, theta, alpha, N, iterations):
    for iter in range(iterations):
        x_transpose = x.transpose()
        hypothesis = np.dot(x, theta)
        errors = hypothesis - y
        cost = np.sum(errors ** 2) / (2 * m)
        
        gradient = np.dot(x_transpose, errors) / m
        # update theta
        theta = theta - alpha * gradient
    return theta


N = len(i)
iterations = 10000
alpha = 0.005

theta = np.ones(N)
theta = gradientDescent(x, y, theta, alpha, N, iterations)

# Determine the capacity for what can be done in the vein of gradient descent on cost here

